In [1]:
import csv
import ast
from collections import defaultdict
from neo4j import GraphDatabase

In [2]:
uri = "bolt://10.2.42.255:7687"  # Change this to your Neo4j instance URI
username = "neo4j"             # Replace with your username
password = "pass12345"          # Replace with your password
driver = GraphDatabase.driver(uri, auth=(username, password))

## Importing entities onto neo4j

In [7]:
def create_graph(entities_file, cooccurrences_file):
    # Create nodes and relationships in Neo4j
    with driver.session() as session:
        print("Conenction Established!")
        # Load entities
        with open(entities_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            for row in csv_reader:
                print(row)
                # Create entity nodes
                session.run("""
                    MERGE (e:Entity {name: $name, type: $type})
                """, name=row['entity'], type=row['type'])

        # Load co-occurrences
        with open(cooccurrences_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            for row in csv_reader:
                # Create relationships between entities
                session.run("""
                    MATCH (e1:Entity {name: $entity1}), (e2:Entity {name: $entity2})
                    MERGE (e1)-[r:CO_OCCURS_WITH]->(e2)
                    ON CREATE SET r.weight = $weight
                    ON MATCH SET r.weight = r.weight + $weight
                """, entity1=row['entity1'], entity2=row['entity2'], weight=int(row['weight']))



In [8]:
# Specify the file paths
entities_file = "entities.csv"
cooccurrences_file = "cooccurrences.csv"

# Create the graph
create_graph(entities_file, cooccurrences_file)

# Close the driver after operations
driver.close()

Conenction Established!
{'entity': 'Kanadia village', 'type': 'GPE'}


ServiceUnavailable: Couldn't connect to 10.2.42.255:7687 (resolved to ()):
Timed out trying to establish connection to ResolvedIPv4Address(('10.2.42.255', 7687))